In [6]:
import os
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image, ImageFont, ImageDraw
import torch

# Hyper-parameters & Variables setting
num_epoch = 50
batch_size = 100
learning_rate = 0.0002
img_size = 28 * 28
num_channel = 1
dir_name = "CGAN_results"

noise_size = 100
hidden_size1 = 256
hidden_size2 = 512
hidden_size3 = 1024

"""
FOR CONDITIONAL GAN
"""
# The number of MNIST's class label is 10
condition_size = 10


# Device setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Now using {} devices".format(device))


# Create a directory for saving samples
if not os.path.exists(dir_name):
    os.makedirs(dir_name)


# Define discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.linear1 = nn.Linear(img_size + condition_size, hidden_size3)
        self.linear2 = nn.Linear(hidden_size3, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size1)
        self.linear4 = nn.Linear(hidden_size1, 1)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.leaky_relu(self.linear1(x))
        x = self.leaky_relu(self.linear2(x))
        x = self.leaky_relu(self.linear3(x))
        x = self.linear4(x)
        x = self.sigmoid(x)
        return x


# Define generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.linear1 = nn.Linear(noise_size + condition_size, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size3)
        self.linear4 = nn.Linear(hidden_size3, img_size)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.linear4(x)
        x = self.tanh(x)
        return x





# Dataset transform setting
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)])

# MNIST dataset setting
MNIST_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True,
                                           transform=transform,
                                           download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=MNIST_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


# Initialize generator/Discriminator
discriminator = Discriminator()
generator = Generator()

# Device setting
discriminator = discriminator.to(device)
generator = generator.to(device)

# Loss function & Optimizer setting
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)




Now using cuda devices


In [7]:

# For checking CGAN's validity in final step
def check_condition(_generator, make_num, epoch):
    test_image = torch.empty(0).to(device)

    for i in range(make_num):
        test_label = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
        test_label_encoded = F.one_hot(test_label, num_classes=10).to(device)

        # create noise(latent vector) 'z'
        _z = torch.randn(10, noise_size).to(device)
        _z_concat = torch.cat((_z, test_label_encoded), 1)

        test_image = torch.cat((test_image, _generator(_z_concat)), 0)



    _result = test_image.reshape(10 * make_num, 1, 28, 28)
    save_image(_result, os.path.join(dir_name, 'CGAN_test_result.png'), nrow=10)
    torch.save(_result, f"save_{epoch}epoch_Generate_image")

"""
Training part
"""
for epoch in range(num_epoch):
    for i, (images, label) in enumerate(data_loader):

        # make ground truth (labels) -> 1 for real, 0 for fake
        real_label = torch.full((batch_size, 1), 1, dtype=torch.float32).to(device)
        fake_label = torch.full((batch_size, 1), 0, dtype=torch.float32).to(device)

        # reshape real images from MNIST dataset
        real_images = images.reshape(batch_size, -1).to(device)

        """
        FOR CONDITIONAL GAN
        """
        # Encode MNIST's label's with 'one hot encoding'
        label_encoded = F.one_hot(label, num_classes=10).to(device)
        # concat real images with 'label encoded vector'
        real_images_concat = torch.cat((real_images, label_encoded), 1)

        # +---------------------+
        # |   train Generator   |
        # +---------------------+

        # Initialize grad
        g_optimizer.zero_grad()
        d_optimizer.zero_grad()

        # make fake images with generator & noise vector 'z'
        z = torch.randn(batch_size, noise_size).to(device)

        """
        FOR CONDITIONAL GAN
        """
        # concat noise vector z with encoded labels
        z_concat = torch.cat((z, label_encoded), 1)
        fake_images = generator(z_concat)
        fake_images_concat = torch.cat((fake_images, label_encoded), 1)

        # Compare result of discriminator with fake images & real labels
        # If generator deceives discriminator, g_loss will decrease
        g_loss = criterion(discriminator(fake_images_concat), real_label)

        # Train generator with backpropagation
        g_loss.backward()
        g_optimizer.step()

        # +---------------------+
        # | train Discriminator |
        # +---------------------+

        # Initialize grad
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()

        # make fake images with generator & noise vector 'z'
        z = torch.randn(batch_size, noise_size).to(device)

        """
        FOR CONDITIONAL GAN
        """
        # concat noise vector z with encoded labels
        z_concat = torch.cat((z, label_encoded), 1)
        fake_images = generator(z_concat)
        fake_images_concat = torch.cat((fake_images, label_encoded), 1)

        # Calculate fake & real loss with generated images above & real images
        fake_loss = criterion(discriminator(fake_images_concat), fake_label)
        real_loss = criterion(discriminator(real_images_concat), real_label)
        d_loss = (fake_loss + real_loss) / 2

        # Train discriminator with backpropagation
        # In this part, we don't train generator
        d_loss.backward()
        d_optimizer.step()

        d_performance = discriminator(real_images_concat).mean()
        g_performance = discriminator(fake_images_concat).mean()

        if (i + 1) % 150 == 0:
            print("Epoch [ {}/{} ]  Step [ {}/{} ]  d_loss : {:.5f}  g_loss : {:.5f}"
                  .format(epoch + 1, num_epoch, i+1, len(data_loader), d_loss.item(), g_loss.item()))

    # print discriminator & generator's performance
    print(" Epock {}'s discriminator performance : {:.2f}  generator performance : {:.2f}"
          .format(epoch + 1, d_performance, g_performance))

    # Save fake images in each epoch
    samples = fake_images.reshape(batch_size, 1, 28, 28)
    save_image(samples, os.path.join(dir_name, 'CGAN_fake_samples{}.png'.format(epoch + 1)))
    # print("label of 'CGAN_fake_samples{}.png' is {}".format(epoch + 1, label))

    # Draw real labels on fake sample images
    # If you got error about this, you can remove lines below
    fake_sample_image = Image.open("{}/CGAN_fake_samples{}.png".format(dir_name, epoch + 1))
    font = ImageFont.truetype("arial.ttf", 17)

    label = label.tolist()
    label = label[:10]
    label = [str(l) for l in label]

    label_text = ", ".join(label)
    label_text = "Conditional GAN -\n" \
                 "first 10 labels in this image :\n" + label_text

    image_edit = ImageDraw.Draw(fake_sample_image)
    image_edit.multiline_text(xy=(15, 300),
                              text=label_text,
                              fill=(0, 255, 255),
                              font=font,
                              stroke_width= 4,
                              stroke_fill=(0, 0, 0))
    fake_sample_image.save("{}/CGAN_fake_samples{}.png".format(dir_name, epoch + 1))

    if (epoch + 1)%5 == 0:
        check_condition(generator, 10, epoch + 1)



Epoch [ 1/50 ]  Step [ 150/600 ]  d_loss : 0.04000  g_loss : 3.18066
Epoch [ 1/50 ]  Step [ 300/600 ]  d_loss : 0.00888  g_loss : 11.15326
Epoch [ 1/50 ]  Step [ 450/600 ]  d_loss : 0.01638  g_loss : 6.12786
Epoch [ 1/50 ]  Step [ 600/600 ]  d_loss : 1.60315  g_loss : 24.10291
 Epock 1's discriminator performance : 0.86  generator performance : 0.00
Epoch [ 2/50 ]  Step [ 150/600 ]  d_loss : 0.01249  g_loss : 8.40562
Epoch [ 2/50 ]  Step [ 300/600 ]  d_loss : 0.01380  g_loss : 6.54789
Epoch [ 2/50 ]  Step [ 450/600 ]  d_loss : 0.07674  g_loss : 19.21470
Epoch [ 2/50 ]  Step [ 600/600 ]  d_loss : 0.12314  g_loss : 22.81644
 Epock 2's discriminator performance : 0.95  generator performance : 0.00
Epoch [ 3/50 ]  Step [ 150/600 ]  d_loss : 0.06174  g_loss : 5.89977
Epoch [ 3/50 ]  Step [ 300/600 ]  d_loss : 0.09581  g_loss : 3.51922
Epoch [ 3/50 ]  Step [ 450/600 ]  d_loss : 0.39433  g_loss : 3.31902
Epoch [ 3/50 ]  Step [ 600/600 ]  d_loss : 0.13550  g_loss : 4.46432
 Epock 3's discrimin